In [5]:
import numpy as np

arr = np.arange(24).reshape(6,4)
idxs = np.array([
    [2, 3],
    [5,1],
    [4, 2]
])

print(arr)
print(arr[idxs[:, 0], idxs[:, 1]])

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]]
[11 21 18]


In [6]:
class Node:pass

def kdtree(pointList, depth=0):
    if not pointList:
        return

    # 深さに応じて軸を選択し、軸が順次選択されるようにする
    k = len(pointList[0]) # 全ての点が同じ次元を持つと仮定
    axis = depth % k

    # 点のリストをソートし、中央値の点を選択する
    pointList.sort(key=lambda x:x[axis])
    median = int(len(pointList)/2) # 中央値を選択

    # ノードを作成し、部分木を構築する
    node = Node()
    node.location = pointList[median]
    node.leftChild = kdtree(pointList[0:median], depth+1)
    node.rightChild = kdtree(pointList[median+1:], depth+1)
    return node

import numpy as np
pointList = np.arange(24).reshape(8,3).tolist()
node = kdtree(pointList, depth=0)
print(node.leftChild)

In [1]:
import numpy as np
from tutlibs.io import Points as io

xyz, colors, data = io.read('../data/bunny_pc.ply')
data['nx'] = data['nx'] - 1
data['ny'] = data['ny'] - 1
data['nz'] = data['nz'] - 1
io.write('../data/bunny_pca.ply', xyz, colors, color_range=[0,255], additional_data=data)


In [2]:
import open3d as o3d
import numpy as np
from tutlibs.visualization import Points as visualizer

print("Testing kdtree in Open3D...")
print("Load a point cloud and paint it gray.")
pcd = o3d.io.read_point_cloud("cloud_bin_0.pcd")
pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
print("Find its 200 nearest neighbors, and paint them blue.")
[k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[1500], 200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]
print("Paint the 1500th point red.")
pcd.colors[1500] = [1, 0, 0]
colors = np.array(pcd.colors)
points = np.asarray(pcd.points)

print(colors.shape)

visualizer.k3d(points, colors)


Testing kdtree in Open3D...
Load a point cloud and paint it gray.
Find its 200 nearest neighbors, and paint them blue.
Paint the 1500th point red.
(3903, 3)


Output()

In [ ]:
import numpy as np
import k3d
import os
from tutlibs.normals import normal_estimation
from tutlibs.nns import k_nearest_neighbors
from tutlibs.utils import grouping

from tutlibs.io import Points as io
from tutlibs.visualization import Points as visualizer

xyz, _, _ = io.read('../data/bunny_pc.ply')

# Estimate normals
k = 15
normals = normal_estimation(xyz, k=k)

normal_action = [1]
idx_color = [0xff0000, 0x00ff00, 0x0000ff]
if 0 in normal_action:
    plot = k3d.plot()
    line_coords = []
    start_index = 0
    end_index = 5
    for normal_index in list(range(len(xyz)))[start_index:end_index]:
        w, v = normals[normal_index]
        w_idx = np.argsort(w).astype(np.int32)
        for i, wi in enumerate(w_idx):
            saki = xyz[normal_index]+v[:, wi]*0.1
            plot += k3d.line([xyz[normal_index], saki], width=0.0010, color=idx_color[i])
            # plot += k3d.text(
            #     '{}, {}'.format(w[wi].tolist(), v[:, wi].tolist()),
            #     position=saki.tolist(),
            #     color=0xff0000,
            #     size=1
            # )
    # visualizer.k3d(xyz, plot=plot)

    idx, _ = k_nearest_neighbors(xyz, xyz, k)
    knn_points = grouping(xyz, idx)
    normal_knn_points = knn_points[start_index:end_index]
    normal_knn_points = normal_knn_points.reshape(len(normal_knn_points)*k, 3)
    normal_knn_colors = np.tile(np.array([[0, 1, 1]]), (len(normal_knn_points), 1))

    visualizer.k3d(np.vstack([xyz, normal_knn_points]),
                   np.vstack([colors, normal_knn_colors]), plot=plot)

if 1 in normal_action:
    normals_colors = normals
    visualizer.k3d(xyz, normals_colors, color_range=[-1, 1])

if 2 in normal_action:
    os.makedirs('outputs/', exist_ok=True)
    io.write('outputs/numpy_normal.ply', xyz, normals_colors)